In [1]:
import urllib.request, json 
import pandas as pd
import os
import functools
import time
import sys
from env import *

In [2]:
pd.set_option('display.max_columns', None)

The following function is taken from https://realpython.com/primer-on-python-decorators/ and is used to measure the runtime of functions.

In [3]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

Important paths definitions

Loading UXtweak dataframe (information about sessions). Dropping unnecessary columns.

In [4]:
uxtweak_df = pd.read_excel(PAIRED_SESSIONS, sheet_name="Sessions")
uxtweak_df = uxtweak_df.dropna(axis = 0, subset="Limesurvey_Seed")
uxtweak_df = uxtweak_df[uxtweak_df['Limesurvey_Seed'] != '-']
uxtweak_df[:5]

,order,status,started at,corrected start at,device,os,browser,location,time taken,Session,Task,Limesurvey_Seed
5,8,completed,2022-03-29 08:50:10,2022-03-29 10:50:10,desktop,Windows,Chrome,SK,00:13:58,gsd4FIhsnwaOIMOtoSqX4geVy,1.0,1445944018
8,11,completed,2022-03-29 09:14:01,2022-03-29 11:14:01,desktop,Windows,Chrome,SK,00:07:08,fjAPfBwmai5If7wWXOgoMldv5,2.0,1262028965
10,13,completed,2022-03-29 09:36:32,2022-03-29 11:36:32,desktop,OS X,Chrome,SK,00:13:04,IKW3ZZ9lcX5Z3vSKEghI73tuI,1.0,1313525205
12,15,completed,2022-03-29 10:59:33,2022-03-29 12:59:33,desktop,Windows,Chrome,SK,00:00:06,UEs1kb8w1KAzuLZMpFE3hzHGX,1.0,1979816098
14,17,completed,2022-03-29 12:01:35,2022-03-29 14:01:35,desktop,Windows,Chrome,SK,00:06:54,Nhkvw5dbEwImQaB1qZNPqm7Qh,2.0,188959783


In [5]:
len(uxtweak_df)

140

In [6]:
uxtweak_df.columns

Index(['order', 'status', 'started at', 'corrected start at', 'device', 'os',
       'browser', 'location', 'time taken', 'Session', 'Task',
       'Limesurvey_Seed'],
      dtype='object')

In [7]:
len(uxtweak_df.columns)

12

Loading Limesurvey dataframe (information about filled forms). Renaming columns.

In [8]:
limesurvey_df = pd.read_excel(ALL_LIMESURVEY_DATA, sheet_name="Limesurvey")
limesurvey_df.drop(['G03Q15. Do poľa nižšie napíšte svoj tip na tému, na ktorú je podľa vás výskum zameraný:'], axis=1, inplace=True)
rename_dict = {
    'Unnamed: 0' : 'order',
    'id. ID odpovede' : 'id', 
    'submitdate. Dátum odoslania' : 'submitdate', 
    'lastpage. Posledná strana' : 'lastpage', 
    'startlanguage. Začať jazyk' : 'startlanguage',
    'seed. Seed' : 'seed', 
    'startdate. Dátum začatia' : 'startdate', 
    'datestamp. Dátum poslednej akcie' : 'datestamp', 
    'interviewtime. Celkový čas' : 'interviewtime'
}
limesurvey_df = limesurvey_df.rename(columns=rename_dict)

demografic_columns = ['G03Q{:02d}'.format(x) for x in range(1, 15)]

demographic_question_regex = 'G03Q(([0-9][1-9])|([1-9][0-9])). [\s\S]*'
limesurvey_df = limesurvey_df.rename(dict(zip(list(limesurvey_df.filter(regex=demographic_question_regex)), demografic_columns)), axis=1, inplace=False)

limesurvey_df[:5]

,order,indicator_fg,indicator_h,id,submitdate,lastpage,startlanguage,kick,seed,startdate,datestamp,total_time,mean_time,total_points,mean_points,mean_points_check,mean_bfi2_e,mean_bfi2_a,mean_bfi2_c,mean_bfi2_n,mean_bfi2_e_sociability,mean_bfi2_e_assertiveness,mean_bfi2_e_energy_level,mean_bfi2_a_compassion,mean_bfi2_a_respectfulness,mean_bfi2_a_trust,mean_bfi2_c_organization,mean_bfi2_c_productiveness,mean_bfi2_c_responsibility,mean_bfi2_n_anxiety,mean_bfi2_n_depression,mean_bfi2_n_emotional_volatility,bfi1,bfi2,rbfi3,rbfi4,bfi6,bfi7,rbfi8,rbfi9,rbfi11,rbfi12,bfi13,bfi14,rbfi16,rbfi17,bfi18,bfi19,bfi21,rbfi22,rbfi23,rbfi24,rbfi26,bfi27,rbfi28,rbfi29,rbfi31,bfi32,bfi33,bfi34,rbfi36,rbfi37,bfi38,bfi39,bfi41,rbfi42,bfi43,rbfi44,bfi46,rbfi47,rbfi48,rbfi49,rbfi51,bfi52,bfi53,bfi54,bfi56,bfi57,rbfi58,bfi59,G03Q01,G03Q02,G03Q03,G03Q04,G03Q05,G03Q06,G03Q07,G03Q08,G03Q09,G03Q10,G03Q11,G03Q12,G03Q13,G03Q14,interviewtime
0,77,1,0,82,2022-04-03 21:15:17,66,sk,0,1237483652,2022-04-03 21:05:36,2022-04-03 21:15:17,362.48,7.551667,236,4.916667,4.916667,5.000000,4.666667,5.000000,5.000000,5.00,5.00,5.0,4.00,5.00,5.00,5.0,5.00,5.00,5.0,5.00,5.0,5,5,5,5,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,Nesúhlasím,Nesúhlasím,Súhlasím,Nesúhlasím,Vôbec nesúhlasím,Ženatý/ Vydatá,46-60,SŠ s maturitou,Plný úväzok,5+,2,Žena,Nie,Nie,608.84
1,71,1,0,76,2022-04-03 17:52:30,66,sk,0,2064152004,2022-04-03 17:46:25,2022-04-03 17:52:30,259.96,5.415833,230,4.791667,4.791667,4.416667,4.750000,5.000000,5.000000,4.50,3.75,5.0,5.00,5.00,4.25,5.0,5.00,5.00,5.0,5.00,5.0,4,5,5,5,3,5,5,5,5,4,5,5,4,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,5,5,5,5,5,5,3,5,5,5,5,4,5,5,Nesúhlasím,Úplne súhlasím,Úplne súhlasím,Súhlasím,Vôbec nesúhlasím,Zadaný/á,16-30,VŠ 1. alebo 2. stupňa,Študent,4,2,Muž,Nie,Nie,392.38
2,82,1,0,87,2022-04-04 13:48:30,66,sk,0,1290792711,2022-04-04 13:32:02,2022-04-04 13:48:30,807.45,16.821875,228,4.750000,4.750000,4.500000,4.666667,5.000000,4.833333,3.75,4.75,5.0,5.00,4.75,4.25,5.0,5.00,5.00,5.0,5.00,4.5,1,5,5,5,4,4,5,5,5,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,5,4,5,5,5,5,5,5,5,5,5,3,5,4,Nesúhlasím,Súhlasím,Úplne súhlasím,Neutrálne,Nesúhlasím,Ženatý/ Vydatá,31-45,VŠ 1. alebo 2. stupňa,Plný úväzok,4,1,Muž,Nie,Nie,1018.24
3,31,1,0,36,2022-03-30 12:52:07,66,sk,0,618380686,2022-03-30 12:39:42,2022-03-30 12:52:07,502.08,10.460000,226,4.708333,4.708333,4.416667,4.916667,4.833333,4.666667,4.75,3.50,5.0,5.00,5.00,4.75,5.0,4.75,4.75,4.5,4.50,5.0,5,5,5,4,4,5,4,4,5,5,5,5,4,5,5,4,4,5,5,4,5,5,5,5,5,5,5,5,4,5,5,5,5,4,5,5,5,5,5,5,2,5,5,5,5,5,4,5,Nesúhlasím,Súhlasím,Súhlasím,Súhlasím,Nesúhlasím,Slobodný/á,16-30,Základné,Študent,4,0 (Nezmenil/a),Muž,Nie,Nie,777.36
4,17,1,0,22,2022-03-30 08:14:40,66,sk,0,1013517210,2022-03-30 08:05:16,2022-03-30 08:14:40,284.97,5.936875,223,4.645833,4.645833,4.583333,4.833333,4.416667,4.750000,4.75,4.50,4.5,4.75,5.00,4.75,4.0,4.75,4.50,4.5,4.75,5.0,5,5,5,5,5,5,4,5,4,5,4,5,5,5,4,4,5,5,5,4,4,5,4,5,4,4,3,4,4,5,5,5,5,4,5,5,5,5,4,5,4,5,5,5,5,5,5,5,Nesúhlasím,Neutrálne,Súhlasím,Úplne súhlasím,Nesúhlasím,Zadaný/á,16-30,SŠ bez maturity,Študent,3,0 (Nezmenil/a),Žena,Nie,Nie,593.02


In [9]:
len(limesurvey_df)

145

In [10]:
limesurvey_df.columns

Index(['order', 'indicator_fg', 'indicator_h', 'id', 'submitdate', 'lastpage',
       'startlanguage', 'kick', 'seed', 'startdate', 'datestamp', 'total_time',
       'mean_time', 'total_points', 'mean_points', 'mean_points_check',
       'mean_bfi2_e', 'mean_bfi2_a', 'mean_bfi2_c', 'mean_bfi2_n',
       'mean_bfi2_e_sociability', 'mean_bfi2_e_assertiveness',
       'mean_bfi2_e_energy_level', 'mean_bfi2_a_compassion',
       'mean_bfi2_a_respectfulness', 'mean_bfi2_a_trust',
       'mean_bfi2_c_organization', 'mean_bfi2_c_productiveness',
       'mean_bfi2_c_responsibility', 'mean_bfi2_n_anxiety',
       'mean_bfi2_n_depression', 'mean_bfi2_n_emotional_volatility', 'bfi1',
       'bfi2', 'rbfi3', 'rbfi4', 'bfi6', 'bfi7', 'rbfi8', 'rbfi9', 'rbfi11',
       'rbfi12', 'bfi13', 'bfi14', 'rbfi16', 'rbfi17', 'bfi18', 'bfi19',
       'bfi21', 'rbfi22', 'rbfi23', 'rbfi24', 'rbfi26', 'bfi27', 'rbfi28',
       'rbfi29', 'rbfi31', 'bfi32', 'bfi33', 'bfi34', 'rbfi36', 'rbfi37',
       'bfi38', 'bf

In [11]:
len(limesurvey_df.columns)

95

Merging UXtweak dataframe and Limesurvey dataframe.

In [12]:
ux_lim = pd.merge(uxtweak_df, limesurvey_df, how='inner', left_on = 'Limesurvey_Seed', right_on = 'seed')

In [13]:
ux_lim[:5]

,order_x,status,started at,corrected start at,device,os,browser,location,time taken,Session,Task,Limesurvey_Seed,order_y,indicator_fg,indicator_h,id,submitdate,lastpage,startlanguage,kick,seed,startdate,datestamp,total_time,mean_time,total_points,mean_points,mean_points_check,mean_bfi2_e,mean_bfi2_a,mean_bfi2_c,mean_bfi2_n,mean_bfi2_e_sociability,mean_bfi2_e_assertiveness,mean_bfi2_e_energy_level,mean_bfi2_a_compassion,mean_bfi2_a_respectfulness,mean_bfi2_a_trust,mean_bfi2_c_organization,mean_bfi2_c_productiveness,mean_bfi2_c_responsibility,mean_bfi2_n_anxiety,mean_bfi2_n_depression,mean_bfi2_n_emotional_volatility,bfi1,bfi2,rbfi3,rbfi4,bfi6,bfi7,rbfi8,rbfi9,rbfi11,rbfi12,bfi13,bfi14,rbfi16,rbfi17,bfi18,bfi19,bfi21,rbfi22,rbfi23,rbfi24,rbfi26,bfi27,rbfi28,rbfi29,rbfi31,bfi32,bfi33,bfi34,rbfi36,rbfi37,bfi38,bfi39,bfi41,rbfi42,bfi43,rbfi44,bfi46,rbfi47,rbfi48,rbfi49,rbfi51,bfi52,bfi53,bfi54,bfi56,bfi57,rbfi58,bfi59,G03Q01,G03Q02,G03Q03,G03Q04,G03Q05,G03Q06,G03Q07,G03Q08,G03Q09,G03Q10,G03Q11,G03Q12,G03Q13,G03Q14,interviewtime
0,8,completed,2022-03-29 08:50:10,2022-03-29 10:50:10,desktop,Windows,Chrome,SK,00:13:58,gsd4FIhsnwaOIMOtoSqX4geVy,1.0,1445944018,1,1,0,6,2022-03-29 11:03:59,66,sk,0,1445944018,2022-03-29 10:50:48,2022-03-29 11:03:59,253.26,5.276250,184,3.833333,3.833333,2.916667,4.416667,4.166667,3.833333,2.25,2.75,3.75,4.75,4.50,4.00,4.25,4.00,4.25,3.25,3.75,4.50,4,5,5,3,4,5,4,3,5,4,4,5,1,5,3,4,1,5,4,4,4,5,4,3,1,4,4,3,2,3,4,4,3,4,4,5,3,5,5,3,4,5,4,4,3,3,5,5,Nesúhlasím,Úplne súhlasím,Úplne súhlasím,Súhlasím,Nesúhlasím,Slobodný/á,16-30,SŠ s maturitou,Študent,2,1,Muž,Nie,Áno,817.88
1,11,completed,2022-03-29 09:14:01,2022-03-29 11:14:01,desktop,Windows,Chrome,SK,00:07:08,fjAPfBwmai5If7wWXOgoMldv5,2.0,1262028965,1,0,1,22,NaN,65,sk,0,1262028965,2022-03-29 11:14:53,2022-03-29 11:21:02,286.00,5.958333,146,3.041667,3.041667,2.666667,3.333333,3.500000,2.666667,2.25,3.00,2.75,3.75,3.75,2.50,3.75,2.75,4.00,1.75,3.25,3.00,3,4,3,2,3,4,2,3,4,3,5,2,2,4,5,2,3,5,2,2,2,3,3,2,2,4,4,2,3,2,4,4,2,2,3,5,2,3,3,1,3,4,3,4,3,2,5,3,Nesúhlasím,Súhlasím,Súhlasím,Neutrálne,Nesúhlasím,Slobodný/á,16-30,SŠ s maturitou,Študent,5+,0 (Nezmenil/a),Žena,Nie,Nie,398.50
2,13,completed,2022-03-29 09:36:32,2022-03-29 11:36:32,desktop,OS X,Chrome,SK,00:13:04,IKW3ZZ9lcX5Z3vSKEghI73tuI,1.0,1313525205,3,1,0,8,2022-03-29 11:49:25,66,sk,0,1313525205,2022-03-29 11:40:05,2022-03-29 11:49:25,325.25,6.776042,148,3.083333,3.083333,2.000000,3.833333,3.750000,2.750000,1.50,2.00,2.50,4.25,4.75,2.50,3.75,3.25,4.25,3.00,2.50,2.75,2,4,2,4,3,5,1,2,2,2,4,1,1,5,4,2,1,5,2,2,2,2,4,2,1,4,4,4,2,4,5,4,4,3,5,4,2,4,5,2,2,5,5,2,2,3,4,4,Nesúhlasím,Úplne súhlasím,Súhlasím,Úplne súhlasím,Vôbec nesúhlasím,Slobodný/á,16-30,SŠ s maturitou,Študent,4,0 (Nezmenil/a),Muž,Nie,Nie,590.25
3,15,completed,2022-03-29 10:59:33,2022-03-29 12:59:33,desktop,Windows,Chrome,SK,00:00:06,UEs1kb8w1KAzuLZMpFE3hzHGX,1.0,1979816098,5,1,0,10,2022-03-29 13:07:38,66,sk,1,1979816098,2022-03-29 13:00:47,2022-03-29 13:07:38,280.60,5.845833,197,4.104167,4.104167,3.833333,4.000000,4.583333,4.000000,4.25,3.50,3.75,4.00,5.00,3.00,4.50,4.50,4.75,3.75,3.50,4.75,5,4,3,3,3,5,4,3,3,2,5,5,4,3,5,4,5,5,5,3,5,5,4,5,4,5,5,4,3,5,5,4,4,3,5,5,4,4,5,4,3,5,4,4,3,2,5,4,Nesúhlasím,Úplne súhlasím,Úplne súhlasím,Súhlasím,Vôbec nesúhlasím,Slobodný/á,16-30,SŠ s maturitou,Študent,2,0 (Nezmenil/a),Muž,Nie,Nie,437.49
4,17,completed,2022-03-29 12:01:35,2022-03-29 14:01:35,desktop,Windows,Chrome,SK,00:06:54,Nhkvw5dbEwImQaB1qZNPqm7Qh,2.0,188959783,2,0,1,23,2022-03-29 14:08:20,66,sk,0,188959783,2022-03-29 14:02:04,2022-03-29 14:08:20,227.94,4.748750,175,3.645833,3.645833,3.166667,4.333333,4.083333,3.000000,2.75,3.00,3.75,4.50,4.25,4.25,3.75,3.75,4.75,3.25,3.00,2.75,4,5,3,3,3,4,3,3,4,5,5,4,3,4,4,3,4,4,4,2,3,4,4,2,1,5,4,3,2,5,4,3,4,4,5,3,3,4,4,4,3,4,4,4,4,4,5,2,Nesúhlasím,Súhlasím,Úplne súhlasím,Súhlasím,Vôbec nesúhlasím,Slobodný/á,16-30,SŠ s maturitou,Študent,4,0 (Nezmenil/a),Žena,Nie,Nie,407.72


Removing flagged sessions.

In [14]:
ux_lim = ux_lim[ux_lim['kick'] == 0]

In [15]:
len(ux_lim)

134

In [16]:
ux_lim.columns

Index(['order_x', 'status', 'started at', 'corrected start at', 'device', 'os',
       'browser', 'location', 'time taken', 'Session',
       ...
       'G03Q06', 'G03Q07', 'G03Q08', 'G03Q09', 'G03Q10', 'G03Q11', 'G03Q12',
       'G03Q13', 'G03Q14', 'interviewtime'],
      dtype='object', length=107)

In [17]:
len(ux_lim.columns)

107

Loading session metrics dataframe (calculated metrics from JSONs).

In [18]:
metrics_df = pd.read_json(SESSION_METRICS_FROM_BAKED_JSON, orient ='records')
metrics_df[:5]

Session  big5_mean_time  big5_mean_duration  \
0  S5WLXErzUWNQqkDKnRC1j53PD      927.687500         1129.958333   
1  gsd4FIhsnwaOIMOtoSqX4geVy     2861.604167         4713.104167   
2  IKW3ZZ9lcX5Z3vSKEghI73tuI     4072.166667         6049.333333   
3  DrTk41pNckxJYIdilpta9qJBT     4632.333333         5442.458333   
4  jcvlt3renH3FwZuO0vUqmgbKt     2819.625000         5388.916667   

   big5_mean_distance  big5_mean_x_axis_distance  big5_mean_y_axis_distance  \
0          687.662935                 539.821777                 352.864583   
1          855.009425                 685.480469                 372.833333   
2         1271.899270                1062.445312                 498.375000   
3         1689.802317                1425.574219                 600.791667   
4          933.318823                 731.521484                 439.187500   

   big5_mean_real_ideal_trajectory_diff  big5_mean_max_deviation  \
0                            315.013898                11.210961   
1                            304.349121                66.593651   
2                            622.502322               179.196994   
3                           1043.191131               234.883818   
4                            414.481089                81.647968   

   big5_mean_velocity  big5_mean_x_axis_vel  big5_mean_y_axis_vel  \
0            0.751742              0.583339              0.395297   
1            0.373892              0.298735              0.170425   
2            0.397325              0.339513              0.150907   
3            0.413681              0.344645              0.154020   
4            0.448751              0.342532              0.226396   

   big5_mean_acc  big5_mean_x_axis_acc  big5_mean_y_axis_acc  \
0       0.000859              0.000656              0.000464   
1       0.000231              0.000180              0.000112   
2       0.000165              0.000143              0.000061   
3       0.000124              0.000102              0.000049   
4       0.000291              0.000214              0.000156   

   big5_mean_auc_diff  big5_mean_clicks  big5_mean_x_flips  big5_mean_y_flips  \
0       106455.128479          2.062500           1.458333           1.604167   
1       110704.204102          2.250000           2.354167           2.645833   
2        97427.068750          2.375000           3.791667           3.208333   
3       122730.759766          2.291667           5.229167           5.895833   
4        95816.069336          2.083333           3.729167           2.895833   

   big5_mean_init_time  big5_mean_react_time  big5_sum_visits  \
0           192.770833            332.395833               49   
1          1531.083333           3692.000000               48   
2          1664.583333           4890.312500               49   
3           524.354167           4230.395833               49   
4          2220.812500           4348.145833               52   

   big5_sum_scrolling  big5_sum_x_flips  big5_sum_y_flips  dem_mean_time  \
0                   0                70                77     968.142857   
1                   0               113               127    2596.928571   
2                   0               182               154    3355.428571   
3                   0               251               283    3403.214286   
4                   0               179               139    3208.642857   

   dem_mean_duration  dem_mean_distance  dem_mean_x_axis_distance  \
0        1172.928571         676.131010                537.631138   
1        4311.571429         774.897241                632.524554   
2        4910.928571        1114.092414                923.705357   
3        4672.142857        1445.463065               1249.064732   
4        5244.500000        1045.206720                876.488839   

   dem_mean_y_axis_distance  dem_mean_real_ideal_trajectory_diff  \
0                349.553571                           259.478873   
1                335.571429               

In [19]:
len(metrics_df)

138

In [20]:
metrics_df.columns

Index(['Session', 'big5_mean_time', 'big5_mean_duration', 'big5_mean_distance',
       'big5_mean_x_axis_distance', 'big5_mean_y_axis_distance',
       'big5_mean_real_ideal_trajectory_diff', 'big5_mean_max_deviation',
       'big5_mean_velocity', 'big5_mean_x_axis_vel',
       ...
       'Page_67-x_axis_acc', 'Page_67-y_axis_acc', 'Page_67-auc_diff',
       'Page_67-clicks', 'Page_67-x_flips', 'Page_67-y_flips',
       'Page_67-visits', 'Page_67-scrolling', 'Page_67-init_time',
       'Page_67-react_time'],
      dtype='object', length=1475)

Merging (UXtweak dataframe and Limesurvey dataframe) with metrics dataframe.

In [21]:
len(metrics_df.columns)

1475

In [22]:
ux_lim_met = pd.merge(ux_lim, metrics_df, how='inner', on='Session')

Show final dataframe.

In [23]:
ux_lim_met[:5]

order_x     status          started at  corrected start at   device  \
0        8  completed 2022-03-29 08:50:10 2022-03-29 10:50:10  desktop   
1       13  completed 2022-03-29 09:36:32 2022-03-29 11:36:32  desktop   
2       17  completed 2022-03-29 12:01:35 2022-03-29 14:01:35  desktop   
3       18  completed 2022-03-29 12:33:11 2022-03-29 14:33:11  desktop   
4       21  completed 2022-03-29 16:40:01 2022-03-29 18:40:01  desktop   

        os  browser location time taken                    Session  Task  \
0  Windows   Chrome       SK   00:13:58  gsd4FIhsnwaOIMOtoSqX4geVy   1.0   
1     OS X   Chrome       SK   00:13:04  IKW3ZZ9lcX5Z3vSKEghI73tuI   1.0   
2  Windows   Chrome       SK   00:06:54  Nhkvw5dbEwImQaB1qZNPqm7Qh   2.0   
3   Ubuntu  Firefox       SK   00:07:19  5HPhIkC2ryz2x5IiTSDGramvs   2.0   
4  Windows   Chrome       SK   00:00:04  v12R8blPbUmMEeidxzOBHB8Sg   2.0   

  Limesurvey_Seed  order_y  indicator_fg  indicator_h  id  \
0      1445944018        1             1            0   6   
1      1313525205        3             1            0   8   
2       188959783        2             0            1  23   
3      2103452617        3             0            1  24   
4      1807918619        4             0            1  25   

            submitdate  lastpage startlanguage  kick        seed  \
0  2022-03-29 11:03:59        66            sk     0  1445944018   
1  2022-03-29 11:49:25        66            sk     0  1313525205   
2  2022-03-29 14:08:20        66            sk     0   188959783   
3  2022-03-29 14:40:25        66            sk     0  2103452617   
4  2022-03-29 18:48:08        66            sk     0  1807918619   

             startdate            datestamp  total_time  mean_time  \
0  2022-03-29 10:50:48  2022-03-29 11:03:59      253.26   5.276250   
1  2022-03-29 11:40:05  2022-03-29 11:49:25      325.25   6.776042   
2  2022-03-29 14:02:04  2022-03-29 14:08:20      227.94   4.748750   
3  2022-03-29 14:33:26  2022-03-29 14:40:25      272.48   5.676667   
4  2022-03-29 18:40:51  2022-03-29 18:48:08      290.53   6.052708   

   total_points  mean_points  mean_points_check  mean_bfi2_e  mean_bfi2_a  \
0           184     3.833333           3.833333     2.916667     4.416667   
1           148     3.083333           3.083333     2.000000     3.833333   
2           175     3.645833           3.645833     3.166667     4.333333   
3           159     3.312500           3.312500     2.500000     3.750000   
4           169     3.520833           3.520833     3.500000     3.916667   

   mean_bfi2_c  mean_bfi2_n  mean_bfi2_e_sociability  \
0     4.166667     3.833333                     2.25   
1     3.750000     2.750000                     1.50   
2     4.083333     3.000000                     2.75   
3     3.416667     3.583333                     1.50   
4     4.416667     2.250000                     3.00   

   mean_bfi2_e_assertiveness  mean_bfi2_e_energy_level  \
0                       2.75                      3.75   
1                       2.00                      2.50   
2                       3.00                      3.75   
3                       3.00                      3.00   
4                       3.50                      4.00   

   mean_bfi2_a_compassion  mean_bfi2_a_respectfulness  mean_bfi2_a_trust  \
0                    4.75                        4.50               4.00   
1                    4.25                        4.75               2.50   
2                    4.50                        4.25               4.25   
3                    3.25                        4.25               3.75   
4                    4.75                        3.75               3.25   

   mean_bfi2_c_organization  mean_bfi2_c_productiveness  \
0                      4.25                        4.00   
1                      3.75                        3.25   
2                      3.75                        3.75   
3                      4.00                        3.25   
4  

In [24]:
len(ux_lim_met)

131

In [25]:
ux_lim_met.columns

Index(['order_x', 'status', 'started at', 'corrected start at', 'device', 'os',
       'browser', 'location', 'time taken', 'Session',
       ...
       'Page_67-x_axis_acc', 'Page_67-y_axis_acc', 'Page_67-auc_diff',
       'Page_67-clicks', 'Page_67-x_flips', 'Page_67-y_flips',
       'Page_67-visits', 'Page_67-scrolling', 'Page_67-init_time',
       'Page_67-react_time'],
      dtype='object', length=1581)

In [26]:
len(ux_lim_met.columns)

1581

Save final dataframe.

In [27]:
result = ux_lim_met.to_json(orient="records")

In [28]:
with open(MERGED_DATA_FROM_BAKED_JSON, 'w') as outfile:
    outfile.write(result)